### Solution 2

In solutions 2, I tried to extract just specific informations needed, such as name, address, amount transactions, etc.

In [1]:
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 
import pandas as pd

In [2]:
# Path of the pdf 
PDF_file = "bankstatementinput.pdf"

In [3]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [4]:
# Store all the pages of the PDF in a variable 
pages = convert_from_path(PDF_file)

In [5]:
# Counter to store images of each page of PDF to image 
image_counter = 1
  
# Iterate through all the pages stored above 
for page in pages: 
  
    # Declaring filename for each page of PDF as JPG 
    # For each page, filename will be: 
    # PDF page 1 -> page_1.jpg 
    # PDF page 2 -> page_2.jpg 
    # PDF page 3 -> page_3.jpg 
    # .... 
    # PDF page n -> page_n.jpg 
    filename = "page_"+str(image_counter)+".jpg"
      
    # Save the image of the page in system 
    page.save(filename, 'JPEG') 
  
    # Increment the counter to update filename 
    image_counter = image_counter + 1

In [6]:
  
''' 
Part #2 - Recognizing text from the images using OCR 
'''

# Variable to get count of total number of pages 
filelimit = image_counter-1

datas = []
# Iterate from 1 to total number of pages 
for i in range(1, filelimit + 1): 
  
    # Set filename to recognize text from 
    # Again, these files will be: 
    # page_1.jpg 
    # page_2.jpg 
    # .... 
    # page_n.jpg 
    filename = "page_"+str(i)+".jpg"
          
    # Recognize the text as string in image using pytesserct 
    text = str(((pytesseract.image_to_string(Image.open(filename)))))
  
    text = text.replace('&\na','').replace('ACCOUNT SUMMARY | Asat','').replace('ACCOUNT DETAILS | Asat','').split('\n')
    
    datas.append(text)

From the code above, I created 2 lists, lists from page 1 and page 2. And then I combined both of the list become AllList

In [7]:
AllList = sum(datas,[])

In [8]:
# Names = []
# Addresses = []
# Accounts = []
# StatDates = []
# Dates = []

Extract specific information needed such as name, address, and transaction details.

In [9]:
custData = {}
for i in range(len(AllList)):
    if 'S/N:' in AllList[i]:
        custData['Name'] = AllList[i+2]
#         Names.append(custData['Name'])
        custData['Address'] = AllList[i+4:i+10]
        custData['Address'] = ' '.join(custData['Address'])
#         Addresses.append(custData['Address'])
        custData['Account'] = AllList[i+22]
#         Accounts.append(custData['Account'])
        custData['Statement Date'] = AllList[i+15]
#         StatDates.append(custData['Statement Date'])
    elif 'Balance Brought Forward' in AllList[i]:
        custData['Transaction Date'] = [AllList[i+3][0:6], AllList[i+5][0:6], AllList[i+7][0:6], AllList[i+9][0:6]]
        custData['Transaction Type'] = [AllList[i+3][7:33], AllList[i+5][7:33], AllList[i+7][7:33], AllList[i+9][7:33]]
        custData['Transaction Amount'] = [AllList[i+3][33:], AllList[i+5][33:], AllList[i+7][33:], AllList[i+9][33:]]
        custData['Transaction Desc'] = [AllList[i+4], AllList[i+6], AllList[i+8], AllList[i+10]]

In [10]:
# I was trying to append and then store it into dataframe but it wasn;t work as I intended, still trying to figure it out.

# pd.DataFrame({"custData['Name']":Names,
#              "custData['Address']":Addresses,
#              "custData['Account']":Accounts,
#              "custData['Statement Date']":StatDates,
#              "custData['Date']":Dates})

Here is the specific output as the assignment stated, I extracted several key informations from the pdfs doc. But This output would not be generic as when the format and layout change the result would probably change 

In [11]:
custData

{'Name': 'JOHN RIEGER',
 'Address': 'KTGIF SINGAPORE PTE. LTD. 26B TEMPLE STREET  #03-00  SINGAPORE 058571',
 'Account': 'DBS Multiplier Account 12-145753-2',
 'Statement Date': '31 Aug 2018',
 'Transaction Date': ['30 Aug', '30 Aug', '31 Aug', '31 Aug'],
 'Transaction Type': ['Point-of-Sale Transaction ',
  'Point-of-Sale Transaction ',
  'Point-of-Sale Transaction ',
  'Point-of-Sale Transaction '],
 'Transaction Amount': ['20.00', '465.00', '26.50', '16.00'],
 'Transaction Desc': ['TRANSIT LINK PTE LTD',
  'S & S LINKERS PTE LTD',
  'GAYATRI RESTAURANT',
  'FOCUS NETWORK AGENCIES (S) PTELTD']}